In [1]:
import pandas as pd
from transformers import pipeline
import nltk

c:\Users\User\.conda\envs\UW\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\User\.conda\envs\UW\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
c:\Users\User\.conda\envs\UW\lib\site-packages\numpy\.libs\libopenblas64__v0.3.23-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"
c:\Users\User\.conda\envs\UW\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
BART_classifier = pipeline('zero-shot-classification', model='facebook/bart-large-mnli')
#mDeBERTa_classifier = pipeline("zero-shot-classification", model="MoritzLaurer/mDeBERTa-v3-base-mnli-xnli")

In [16]:
#df=pd.read_csv('india-news-headlines.csv')
df2 = pd.read_csv('datasets/US_stocks.csv')

df2.drop(['Ticker', 'Company Name', 'Comment'], inplace=True, axis=1)

df2_sector = df2.drop(['Industry Group', 'Industry', 'Sub-Industry'], axis=1)
df2_industry_group = df2.drop(['Sector', 'Industry', 'Sub-Industry'], axis=1)
df2_industry = df2.drop(['Industry Group', 'Sector', 'Sub-Industry'], axis=1)
df2_sub_industry = df2.drop(['Industry Group', 'Industry', 'Sector'], axis=1)

#df2.head(n=10)
#df2_sector.head(n=10)
#df2_industry_group.head(n=10)
#df2_industry.head(n=10)
#df2_sub_industry.head(n=10)
#df2_sector.values[0]

num_sector=len(df2_sector['Sector'].unique())
num_industry_group=len(df2_industry_group['Industry Group'].unique())
num_industry=len(df2_industry['Industry'].unique())
num_sub_industry=len(df2_sub_industry['Sub-Industry'].unique())

print(f'Sectors:{num_sector}\nIndustry Groups:{num_industry_group}\nIndustries:{num_industry}\nSub-Industries:{num_sub_industry}')

Sectors:11
Industry Groups:24
Industries:68
Sub-Industries:155


In [22]:
#Predict Sector with 1st version BART

labels = df2_sector['Sector'].unique()
text = df2_sector['Description'].head(n=3)

hypothesis_template = 'This text is about {}.'
predicted_labels=[]
for sentence in text:

    prediction = BART_classifier(sentence, labels, hypothesis_template=hypothesis_template, multi_label=True)
    
    predicted_labels.append(prediction['labels'][0])

In [27]:
#Text tests
from nlp_utils import *

df2_sector['Description'].head(n=2)
lst=bart(df2_sector['Description'].head(n=2), 250)



No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


AttributeError: 'BartTokenizerFast' object has no attribute 'max_len'

In [ ]:
#Text cleanning

In [ ]:
#Predict Sector with 2nd version BART

from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("AyoubChLin/Bart-MNLI-CNN_news")

model = AutoModelForSequenceClassification.from_pretrained("AyoubChLin/Bart-MNLI-CNN_news")
classifier = pipeline(
    "zero-shot-classification",
    model=model,
    tokenizer=tokenizer,
    device=0
)

In [28]:
print(predicted_labels)
print(df2_sector.head(n=3))

['Industrials', 'Industrials', 'Industrials']
                                         Description            Sector
0  Corteva, Inc. is a global provider of seed and...         Materials
1  Alico, Inc. is an agribusiness and land manage...  Consumer Staples
2  Limoneira Company is primarily an agribusiness...  Consumer Staples
